## PandasDataFrameOutputParser
* Pandas DataFrame과 LLM을 연동하여 데이터를 쉽게 조회하고 분석할 수 있도록 돕는 출력 파서
* 조화된 데이터에 대한 자연어 질의를 수행하고, 결과를 Pandas DataFrame 형식으로 반환할 수 있습니다.

### PandasDataFrameOutputParser의 장점

- **자연어 기반 데이터 탐색**
    - SQL 없이도 **자연어로 데이터 조회 가능**
    - 데이터 분석 초보자도 쉽게 활용 가능
- **Pandas DataFrame과의 직접적인 연동**
    - LLM을 이용해 데이터프레임에서 필요한 정보만 **효율적으로 추출**
    - 기존 Pandas 기능과 **유연하게 결합 가능**
- **데이터 가공 및 분석 효율화**
    - 반환된 DataFrame을 바로 **분석, 시각화, 모델링 등에 활용 가능**

In [8]:
import pandas as pd
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import re
import os

import sys
sys.path.append(os.path.abspath(os.path.join(os.pardir, "public")))
from model import getModel

In [9]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [10]:
# OpenAI 모델 초기화
# model = getModel()
model = ChatOpenAI(temperature=0, model_name="gpt-4o")

In [11]:
df = pd.read_csv("../csv/titanic.csv")
# print(df.columns)

In [12]:
# Pandas DataFrame Output Parser 설정
parser = PandasDataFrameOutputParser(dataframe=df)

In [13]:
# 형식 지침 출력
format_instructions = parser.get_format_instructions()
# print("Format Instructions:\n", format_instructions)

In [14]:
# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template=""" 
    You are a helpful assistant that interacts with a Pandas DataFrame.
    The DataFrame contains the following columns: {columns}.
    
    Your task is to answer the user's query by generating a command in the following format:
    {format_instructions}
    
    User Query: {query}    
    """,
    input_variables=["query"],
    partial_variables={
        "format_instructions": format_instructions,
        "columns": ", ".join(df.columns)
    },
)
print(prompt.partial_variables['columns'])

Survived, Pclass, Name, Sex, Age, Siblings/Spouses Aboard, Parents/Children Aboard, Fare


In [15]:
# 체인 생성
chain = prompt | model | parser

# 모델 응답 받기
try:
    # **Name 열을 표시하십시오.**
    print('Name 컬럼 출력')
    df_query = "Show the Name column"

    parser_output = chain.invoke({"query": df_query})
    print(type(parser_output))
    print(parser_output)

    # **첫번째 행을 표시하십시오.**
    print('첫번째 행 출력')
    df_query2 = "Show first row"

    parser_output2 = chain.invoke({"query": df_query2})
    print(parser_output2)

except Exception as e:
    print(f"오류 발생: {e}")

Name 컬럼 출력
<class 'dict'>
{'Name': 0                                 Mr. Owen Harris Braund
1      Mrs. John Bradley (Florence Briggs Thayer) Cum...
2                                  Miss. Laina Heikkinen
3            Mrs. Jacques Heath (Lily May Peel) Futrelle
4                                Mr. William Henry Allen
                             ...                        
882                                 Rev. Juozas Montvila
883                          Miss. Margaret Edith Graham
884                       Miss. Catherine Helen Johnston
885                                 Mr. Karl Howell Behr
886                                   Mr. Patrick Dooley
Name: Name, Length: 887, dtype: object}
첫번째 행 출력
오류 발생: Unsupported request type '```
row'.                         Please check the format instructions.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 


In [16]:
import pandas as pd
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_core.prompts import PromptTemplate

model = ChatOpenAI(temperature=0, model_name="gpt-4o")

# 응답 스키마 정의
response_schemas = [
    ResponseSchema(name="data", description="A list of dictionaries representing table rows."),
]

# Output Parser 설정
parser = StructuredOutputParser.from_response_schemas(response_schemas)


# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template="""
    You are an AI assistant that generates tabular data. 
    You must return the data in JSON format that follows this schema:
    
    {format_instructions}
        
    **User Query:**
    {query}
    """,
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)


In [17]:

# 체인 생성 (프롬프트 → 모델 → OutputParser)
chain = prompt | model | parser

# 실행 함수
def generate_dataframe(user_query):
    try:
        # 모델 호출
        json_response = chain.invoke({"query": user_query})

        # 모델이 반환한 JSON을 Pandas DataFrame으로 변환
        df = pd.DataFrame(json_response["data"])

        # 결과 출력
        print("\n🔹 Generated DataFrame:\n")
        return df

    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        return None

In [18]:
# [예제 1] 2024년 상반기 서울 아파트 평균 매매 가격 데이터 생성
print('2024년 하반기 서울 아파트 평균 매매 가격 데이터 생성')
df_seoul_housing = generate_dataframe(
    "Create a dataset of the average apartment sale prices in Seoul for the second half of 2024 with columns: District (구), Average Price (in KRW), Number of Transactions, and Year-over-Year Change (%)."
)
print(df_seoul_housing.shape)
df_seoul_housing

2024년 하반기 서울 아파트 평균 매매 가격 데이터 생성

🔹 Generated DataFrame:

(8, 4)


,District,Average Price,Number of Transactions,Year-over-Year Change (%)
0,Gangnam-gu,1500000000,1200,3.5
1,Jongno-gu,950000000,800,2.1
2,Mapo-gu,1100000000,950,4.0
3,Songpa-gu,1300000000,1100,3.8
4,Yongsan-gu,1400000000,700,2.5
5,Seocho-gu,1450000000,1000,3.2
6,Gwanak-gu,800000000,600,1.8
7,Seodaemun-gu,900000000,750,2.9


In [19]:
print('한국 5대 편의점 브랜드별 2024년 매출 및 점포 수')
# [예제 3] 한국 5대 편의점 브랜드별 2024년 매출 및 점포 수
df_korean_convenience_stores = generate_dataframe(
    "Create a dataset of the top 5 convenience store brands in Korea in 2024 with columns: Brand Name, Number of Stores, Total Revenue (in billion KRW), and Market Share (%)."
)
df_korean_convenience_stores.head()

한국 5대 편의점 브랜드별 2024년 매출 및 점포 수

🔹 Generated DataFrame:



,Brand Name,Number of Stores,Total Revenue (in billion KRW),Market Share (%)
0,CU,15000,5000,35.0
1,GS25,14000,4800,33.5
2,7-Eleven,10000,3000,20.0
3,Emart24,5000,1500,7.5
4,Ministop,3000,700,4.0
